# Heat Balance Controller Success Factor Monitoring

This notebook provides comprehensive monitoring and validation of the Heat Balance Controller implementation.

## Success Criteria
- **Indoor temperature variance reduced by >50%** compared to baseline
- **Oscillation frequency eliminated** (no more than 1 direction change per 4-hour period)
- **Energy efficiency maintained or improved** compared to previous control methods

## Monitoring Capabilities
1. **Temperature Variance Analysis** - Before/after Heat Balance Controller comparison
2. **Oscillation Detection** - Track direction changes over 4-hour periods
3. **Energy Efficiency Metrics** - Comparative analysis with baseline
4. **Mode Distribution Tracking** - CHARGING/BALANCING/MAINTENANCE usage patterns
5. **Trajectory Prediction Accuracy** - Validate 4-hour forecasting performance

In [ ]:
# Import required libraries
import sys
sys.path.append('../')
from notebook_imports import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📊 Heat Balance Controller Monitoring Notebook Loaded")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 1. Configuration and Data Loading

In [ ]:
# Load configuration
config = load_config()

# Connect to InfluxDB
influx = influx_service.InfluxService(
    url=config.influx_url,
    token=config.influx_token,
    org=config.influx_org,
    bucket=config.influx_bucket
)

print("✅ Configuration loaded")
print(f"🔗 Connected to InfluxDB: {config.influx_url}")
print(f"🎯 Heat Balance Mode: {config.heat_balance_mode}")
print(f"⚙️ Controller Thresholds: Charging={config.charging_mode_threshold}°C, Maintenance={config.maintenance_mode_threshold}°C")

## 2. Data Collection Functions

In [ ]:
def get_heat_balance_data(hours_back=168):
    """Get Heat Balance Controller operational data"""
    end_time = datetime.now()
    start_time = end_time - timedelta(hours=hours_back)
    
    # Query for ML heating state and modes
    query = f'''
    from(bucket: "{config.influx_bucket}")
      |> range(start: {start_time.isoformat()}Z, stop: {end_time.isoformat()}Z)
      |> filter(fn: (r) => r._measurement == "state" and 
                 (r.entity_id == "sensor.ml_heating_state" or
                  r.entity_id == "sensor.ml_heating_mode" or
                  r.entity_id == "sensor.ml_model_confidence" or
                  r.entity_id == "sensor.ml_model_mae" or
                  r.entity_id == "sensor.ml_model_rmse"))
      |> filter(fn: (r) => r._field == "value")
      |> pivot(rowKey:["_time"], columnKey: ["entity_id"], valueColumn: "_value")
    '''
    
    try:
        result = influx.client.query_api().query_data_frame(query)
        if not result.empty:
            result['_time'] = pd.to_datetime(result['_time'])
            result = result.set_index('_time')
            return result
    except Exception as e:
        print(f"⚠️ Error querying Heat Balance data: {e}")
    
    return pd.DataFrame()

def get_temperature_data(hours_back=168):
    """Get temperature data for variance analysis"""
    end_time = datetime.now()
    start_time = end_time - timedelta(hours=hours_back)
    
    entities = [
        config.indoor_temp_entity_id,
        config.target_indoor_temp_entity_id,
        config.actual_outlet_temp_entity_id,
        config.target_outlet_temp_entity_id
    ]
    
    query = f'''
    from(bucket: "{config.influx_bucket}")
      |> range(start: {start_time.isoformat()}Z, stop: {end_time.isoformat()}Z)
      |> filter(fn: (r) => r._measurement == "state" and 
                 (r.entity_id == "{entities[0]}" or
                  r.entity_id == "{entities[1]}" or
                  r.entity_id == "{entities[2]}" or
                  r.entity_id == "{entities[3]}"))
      |> filter(fn: (r) => r._field == "value")
      |> pivot(rowKey:["_time"], columnKey: ["entity_id"], valueColumn: "_value")
    '''
    
    try:
        result = influx.client.query_api().query_data_frame(query)
        if not result.empty:
            result['_time'] = pd.to_datetime(result['_time'])
            result = result.set_index('_time')
            # Convert to numeric
            for col in result.columns:
                result[col] = pd.to_numeric(result[col], errors='coerce')
            return result
    except Exception as e:
        print(f"⚠️ Error querying temperature data: {e}")
    
    return pd.DataFrame()

print("✅ Data collection functions defined")

## 3. Success Factor Analysis Functions

In [ ]:
def analyze_temperature_variance(temp_data, window_hours=24):
    """Analyze indoor temperature variance over time windows"""
    if temp_data.empty or config.indoor_temp_entity_id not in temp_data.columns:
        print("⚠️ No indoor temperature data available")
        return None
    
    indoor_temp = temp_data[config.indoor_temp_entity_id].dropna()
    
    # Calculate rolling variance
    rolling_variance = indoor_temp.rolling(f'{window_hours}H').var()
    
    # Calculate overall statistics
    variance_stats = {
        'overall_variance': indoor_temp.var(),
        'overall_std': indoor_temp.std(),
        'mean_rolling_variance': rolling_variance.mean(),
        'max_rolling_variance': rolling_variance.max(),
        'min_rolling_variance': rolling_variance.min()
    }
    
    return variance_stats, rolling_variance

def detect_oscillations(temp_data, window_hours=4, threshold=0.5):
    """Detect temperature oscillations (direction changes)"""
    if temp_data.empty or config.actual_outlet_temp_entity_id not in temp_data.columns:
        print("⚠️ No outlet temperature data available")
        return None
    
    outlet_temp = temp_data[config.actual_outlet_temp_entity_id].dropna()
    
    # Calculate temperature changes (derivatives)
    temp_changes = outlet_temp.diff()
    
    # Identify direction changes (sign changes in derivative)
    direction_changes = (temp_changes.shift(1) * temp_changes < 0) & (abs(temp_changes) > threshold)
    
    # Count oscillations per window
    window_size = f'{window_hours}H'
    oscillations_per_window = direction_changes.rolling(window_size).sum()
    
    oscillation_stats = {
        'total_direction_changes': direction_changes.sum(),
        'hours_analyzed': len(outlet_temp) / 60,  # Assuming 1-minute resolution
        'avg_oscillations_per_4h': oscillations_per_window.mean(),
        'max_oscillations_per_4h': oscillations_per_window.max(),
        'periods_with_excess_oscillations': (oscillations_per_window > 1).sum()
    }
    
    return oscillation_stats, oscillations_per_window

def analyze_mode_distribution(hb_data):
    """Analyze Heat Balance Controller mode usage"""
    if hb_data.empty or 'sensor.ml_heating_mode' not in hb_data.columns:
        print("⚠️ No Heat Balance Controller mode data available")
        return None
    
    mode_data = hb_data['sensor.ml_heating_mode'].dropna()
    
    # Count mode usage
    mode_counts = mode_data.value_counts()
    mode_percentages = mode_data.value_counts(normalize=True) * 100
    
    mode_stats = {
        'mode_counts': mode_counts.to_dict(),
        'mode_percentages': mode_percentages.to_dict(),
        'total_samples': len(mode_data),
        'unique_modes': mode_data.unique().tolist()
    }
    
    return mode_stats

print("✅ Analysis functions defined")

## 4. Data Loading and Initial Analysis

In [ ]:
# Load data for the last week
print("📊 Loading Heat Balance Controller data...")
hb_data = get_heat_balance_data(hours_back=168)
temp_data = get_temperature_data(hours_back=168)

print(f"📈 Heat Balance data points: {len(hb_data)}")
print(f"🌡️ Temperature data points: {len(temp_data)}")

if not hb_data.empty:
    print("\n📋 Available Heat Balance metrics:")
    for col in hb_data.columns:
        print(f"  - {col}")

if not temp_data.empty:
    print("\n🌡️ Available temperature metrics:")
    for col in temp_data.columns:
        print(f"  - {col}")

## 5. Success Factor 1: Temperature Variance Analysis

In [ ]:
# Analyze temperature variance
print("🎯 Success Factor 1: Indoor Temperature Variance Analysis")
print("Target: >50% reduction in temperature variance")

variance_result = analyze_temperature_variance(temp_data)

if variance_result:
    variance_stats, rolling_variance = variance_result
    
    print(f"\n📊 Temperature Variance Statistics:")
    print(f"  Overall Variance: {variance_stats['overall_variance']:.4f}°C²")
    print(f"  Overall Std Dev: {variance_stats['overall_std']:.4f}°C")
    print(f"  Mean 24h Variance: {variance_stats['mean_rolling_variance']:.4f}°C²")
    print(f"  Max 24h Variance: {variance_stats['max_rolling_variance']:.4f}°C²")
    print(f"  Min 24h Variance: {variance_stats['min_rolling_variance']:.4f}°C²")
    
    # Plot temperature variance over time
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
    
    # Plot indoor temperature
    indoor_temp = temp_data[config.indoor_temp_entity_id].dropna()
    ax1.plot(indoor_temp.index, indoor_temp.values, alpha=0.7, linewidth=1)
    ax1.set_title('Indoor Temperature Over Time')
    ax1.set_ylabel('Temperature (°C)')
    ax1.grid(True)
    
    # Plot rolling variance
    ax2.plot(rolling_variance.index, rolling_variance.values, color='red', alpha=0.8)
    ax2.set_title('24-Hour Rolling Temperature Variance')
    ax2.set_ylabel('Variance (°C²)')
    ax2.set_xlabel('Time')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # TODO: Compare with baseline (pre-Heat Balance Controller)
    print("\n⚠️ Note: To calculate variance reduction, baseline data is needed.")
    print("   Collect data from before Heat Balance Controller implementation.")
else:
    print("❌ Unable to analyze temperature variance - insufficient data")

## 6. Success Factor 2: Oscillation Detection

In [ ]:
# Detect oscillations
print("🎯 Success Factor 2: Oscillation Frequency Analysis")
print("Target: ≤1 direction change per 4-hour period")

oscillation_result = detect_oscillations(temp_data, window_hours=4, threshold=0.5)

if oscillation_result:
    oscillation_stats, oscillations_per_window = oscillation_result
    
    print(f"\n📊 Oscillation Statistics:")
    print(f"  Total Direction Changes: {oscillation_stats['total_direction_changes']}")
    print(f"  Hours Analyzed: {oscillation_stats['hours_analyzed']:.1f}")
    print(f"  Avg Oscillations per 4h: {oscillation_stats['avg_oscillations_per_4h']:.2f}")
    print(f"  Max Oscillations per 4h: {oscillation_stats['max_oscillations_per_4h']:.0f}")
    print(f"  Periods with >1 Oscillation: {oscillation_stats['periods_with_excess_oscillations']}")
    
    # Success evaluation
    success = oscillation_stats['avg_oscillations_per_4h'] <= 1.0
    print(f"\n🎯 Oscillation Success: {'✅ PASSED' if success else '❌ FAILED'}")
    
    # Plot oscillation analysis
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
    
    # Plot outlet temperature
    outlet_temp = temp_data[config.actual_outlet_temp_entity_id].dropna()
    ax1.plot(outlet_temp.index, outlet_temp.values, alpha=0.7, linewidth=1)
    ax1.set_title('Outlet Temperature Over Time')
    ax1.set_ylabel('Temperature (°C)')
    ax1.grid(True)
    
    # Plot oscillations per 4-hour window
    ax2.plot(oscillations_per_window.index, oscillations_per_window.values, color='orange')
    ax2.axhline(y=1, color='red', linestyle='--', alpha=0.7, label='Target Threshold (1)')
    ax2.set_title('Direction Changes per 4-Hour Window')
    ax2.set_ylabel('Number of Oscillations')
    ax2.set_xlabel('Time')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ Unable to detect oscillations - insufficient data")

## 7. Success Factor 3: Mode Distribution Analysis

In [ ]:
# Analyze mode distribution
print("🎯 Heat Balance Controller Mode Analysis")
print("Understanding CHARGING/BALANCING/MAINTENANCE mode usage")

mode_stats = analyze_mode_distribution(hb_data)

if mode_stats:
    print(f"\n📊 Mode Distribution Statistics:")
    print(f"  Total Samples: {mode_stats['total_samples']}")
    print(f"  Unique Modes: {mode_stats['unique_modes']}")
    
    print(f"\n📈 Mode Usage:")
    for mode, count in mode_stats['mode_counts'].items():
        percentage = mode_stats['mode_percentages'].get(mode, 0)
        print(f"  {mode}: {count} samples ({percentage:.1f}%)")
    
    # Plot mode distribution
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Pie chart
    modes = list(mode_stats['mode_counts'].keys())
    counts = list(mode_stats['mode_counts'].values())
    ax1.pie(counts, labels=modes, autopct='%1.1f%%', startangle=90)
    ax1.set_title('Mode Distribution')
    
    # Time series
    if 'sensor.ml_heating_mode' in hb_data.columns:
        mode_data = hb_data['sensor.ml_heating_mode'].dropna()
        # Create numerical representation for plotting
        mode_mapping = {mode: i for i, mode in enumerate(mode_data.unique())}
        mode_numeric = mode_data.map(mode_mapping)
        
        ax2.scatter(mode_numeric.index, mode_numeric.values, alpha=0.6, s=10)
        ax2.set_title('Mode Usage Over Time')
        ax2.set_ylabel('Mode')
        ax2.set_xlabel('Time')
        ax2.set_yticks(list(mode_mapping.values()))
        ax2.set_yticklabels(list(mode_mapping.keys()))
        ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ Unable to analyze mode distribution - insufficient data")

## 8. Performance Metrics Summary

In [ ]:
# Display ML model performance metrics
print("🎯 ML Model Performance Summary")

if not hb_data.empty:
    # Get latest performance metrics
    performance_metrics = {}
    
    for metric in ['sensor.ml_model_confidence', 'sensor.ml_model_mae', 'sensor.ml_model_rmse']:
        if metric in hb_data.columns:
            latest_value = hb_data[metric].dropna().iloc[-1] if not hb_data[metric].dropna().empty else None
            performance_metrics[metric.replace('sensor.ml_model_', '')] = latest_value
    
    print(f"\n📊 Current Model Performance:")
    for metric, value in performance_metrics.items():
        if value is not None:
            if metric == 'confidence':
                print(f"  {metric.capitalize()}: {value:.3f} ({value*100:.1f}%)")
            else:
                print(f"  {metric.upper()}: {value:.3f}°C")
        else:
            print(f"  {metric.capitalize()}: No data")
    
    # Plot performance over time
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    metrics_to_plot = [
        ('sensor.ml_model_confidence', 'Model Confidence'),
        ('sensor.ml_model_mae', 'Mean Absolute Error (°C)'),
        ('sensor.ml_model_rmse', 'Root Mean Square Error (°C)'),
        ('sensor.ml_heating_state', 'Heating State Code')
    ]
    
    for i, (metric, title) in enumerate(metrics_to_plot):
        ax = axes[i//2, i%2]
        if metric in hb_data.columns:
            data = hb_data[metric].dropna()
            if not data.empty:
                if metric == 'sensor.ml_heating_state':
                    ax.scatter(data.index, data.values, alpha=0.6, s=10)
                else:
                    ax.plot(data.index, data.values, alpha=0.8)
                ax.set_title(title)
                ax.grid(True)
            else:
                ax.text(0.5, 0.5, 'No Data', ha='center', va='center', transform=ax.transAxes)
                ax.set_title(f'{title} (No Data)')
        else:
            ax.text(0.5, 0.5, 'Metric Not Found', ha='center', va='center', transform=ax.transAxes)
            ax.set_title(f'{title} (Not Available)')
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ No Heat Balance Controller performance data available")

## 9. Success Criteria Evaluation

In [ ]:
# Overall success evaluation
print("🎯 HEAT BALANCE CONTROLLER SUCCESS EVALUATION")
print("=" * 60)

success_criteria = {
    'Temperature Variance Reduction >50%': '⚠️ NEEDS BASELINE DATA',
    'Oscillation Elimination (≤1 per 4h)': '⚠️ DATA DEPENDENT',
    'Energy Efficiency Maintained': '⚠️ NEEDS BASELINE DATA'
}

# Update based on available analysis
if oscillation_result:
    oscillation_success = oscillation_stats['avg_oscillations_per_4h'] <= 1.0
    success_criteria['Oscillation Elimination (≤1 per 4h)'] = '✅ PASSED' if oscillation_success else '❌ FAILED'

print("\n📊 Success Criteria Status:")
for criterion, status in success_criteria.items():
    print(f"  {criterion}: {status}")

print("\n📋 Recommendations:")
print("  1. 📈 Collect baseline data from pre-Heat Balance Controller period")
print("  2. ⚡ Implement energy consumption monitoring integration")
print("  3. 📊 Set up automated daily/weekly success factor reporting")
print("  4. 🔄 Create comparative analysis with heat curve control")
print("  5. 📝 Document optimal mode threshold configurations")

print("\n🎉 Heat Balance Controller Monitoring Complete!")
print(f"📅 Analysis completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 10. Future Enhancement Tracking

### 🚀 Planned Enhancements

1. **Forecast Weighting System**
   - Weight recent forecasts higher than distant ones
   - Implement time-decay weighting for trajectory prediction
   - Status: Planned for future development

2. **Adaptive Trajectory Horizon**
   - Longer prediction for unstable conditions
   - Dynamic adjustment of TRAJECTORY_STEPS based on system stability
   - Status: Planned for future development

3. **Machine Learning Trajectory Scoring**
   - Learn optimal stability weights from historical performance
   - Adaptive OSCILLATION_PENALTY_WEIGHT and FINAL_DESTINATION_WEIGHT
   - Status: Planned for future development

### 📊 Success Factor Monitoring Evolution

This notebook provides the foundation for continuous Heat Balance Controller validation. Future enhancements will include:
- Automated baseline comparison
- Energy efficiency integration
- Real-time success factor alerts
- A/B testing framework for parameter optimization